### TODO
* Read all files into the list
* Check it out
* Choose files one by one
     * get info: platform, channels, business units
     * 

In [72]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import scipy
import scipy.stats as stat
import xlsxwriter
from pathlib import Path

In [2]:
time_window_regr = 12 # number of weeks for regression

In [259]:
list_files = os.listdir("./input")
list_files

['cocochocosk.csv',
 'eyerim.csv',
 'fitplus.csv',
 'hartman.csv',
 'hptronic.csv',
 'istepcesro.csv',
 'ketodiet.csv',
 'krmimkvalitne.csv',
 'muziker.csv',
 'nafigatecosmetics.csv',
 'packwaysro.csv',
 'selsey.csv',
 'semahead.csv',
 'skodaautodigilab.csv',
 'superbet.csv',
 'superzoo.csv']

In [83]:
mypath = Path().absolute()
folder = 'output\\'
img_folder = folder + 'img\\'

In [82]:
folder

'output\\'

In [4]:
# execute for all files
for i in list_files:
    pass

include image if regression available
add only new lines if it need (different numbers for raw with images and blank lines 

In [5]:
# Data type description for dataset columns

DateTimeID = ['periodStartDate']

usecols = [
    "account",
    "adCampaign", "platformCode","currencyCode", "channelCode", "marketingInvestment",
    "impressions", "clicks", "visits", "conversions", "deliveries",
    "netRevenue", "grossProfit", "adGroup", "periodStartDate","businessUnit"
          ]

dtype = {
    'account': str,
    'adCampaign' : str,
    'platformCode' : str,
    'businessUnit':str,
    'currencyCode':str,
    'channelCode' : str,
    'impressions' : int,
    'clicks' : int,
    'visits' : int,
    'conversions' : int,
    'deliveries' : int,
    'netRevenue' : float,
    'grossProfit' : float,
    'adGroup' : str,
    'marketingInvestment': float
}

In [6]:
# function for genetic algorithm to minimize (sum of squared error)
def sumOfSquaredError(parameterTuple):
    function = log_f # it's bad staff #TODO
    warnings.filterwarnings("ignore") # do not print warnings by genetic algorithm
    val = function(t_train, *parameterTuple)
    return np.sum((y_train - val) ** 2.0)

In [7]:
def select_agg_resample_df(df, index, granularity, use_nan):
    """Function for select and group  data in dataframe
    input: pandas dataframe, name of index, granularity['week','month'] default as in dataframe
    return: aggregated dataframe with extra column with indexes [ROI, CPI, CPC]"""
    group_df = df.groupby(['periodStartDate']).agg({
    'netRevenue': 'sum',
    'marketingInvestment': 'sum', 
    'visits': 'sum', 
    'conversions': 'sum',
    'deliveries': 'sum', 
    'impressions': 'sum', 
    'clicks': 'sum', 
    'grossProfit': 'sum' })   

    if granularity == 'week':
        group_agg_df = group_df.resample('W-MON').agg('sum')
    elif granularity == 'month':
        group_agg_df = group_df.resample('M', convention='end').agg('sum')
    elif granularity == 'day':
        group_agg_df = group_df
    else:
        raise ValueError("Incorrect aggregation period, shuld be 'day', 'week' or 'month'")

    if use_nan == False:
        group_agg_df.fillna(0)
    else:
        pass
    return group_agg_df

In [54]:
# function for search initial value for regression parameters
from scipy.optimize import differential_evolution
import warnings
def generate_Initial_Parameters(t_train, y_train,function):
    # min and max used for bounds
    maxX = max(t_train)
    minX = min(t_train)
    maxY = max(y_train)
    minY = min(y_train)
    maxXY = max(maxX, maxY)

    parameterBounds = []
    parameterBounds.append([-maxXY, maxXY]) # seach bounds for a
    parameterBounds.append([-maxXY, maxXY]) # seach bounds for b
    parameterBounds.append([-maxXY, maxXY]) # seach bounds for c

    # "seed" the numpy random number generator for repeatable results
    result = differential_evolution(sumOfSquaredError, parameterBounds, seed=3)
    return result.x

In [55]:
def regression_calc(df, function):
    """Calculate paramiters of regression function
    input: pandas dataframe [x,y]
    output: list[paramiters], list[std diviation err of paramiters]"""
    from scipy.optimize import curve_fit
    t_train = df[df.columns[0]]  #
    y_train = df[df.columns[1]]
    geneticParameters = generate_Initial_Parameters(t_train, y_train, function)
    popt, pcov = curve_fit(function, t_train, y_train, geneticParameters)
    #popt, pcov = curve_fit(function, t_train, y_train)
    error  = np.sqrt(np.diag(pcov))
    return popt, error

In [56]:
def filter_df(df, alpha):
    return df.ewm(alpha=alpha, adjust=False).mean()

In [57]:
# regression functions definition
def log_f(x, a, b, c):
    return a * (1-np.exp(-x/b))+ c  #a * (1 - np.exp((x/b)))

def line_f(x, a, b):
    return a * x + b

def sine_f(x, a, b):
    return a * np.sin(b * x)

In [130]:
def save_chart(sorted_df, name):
    fig, ax = plt.subplots(figsize=(6, 4))
    fig.subplots_adjust(bottom=0.15, left=0.2)
    x = np.linspace(0,sorted_df['marketingInvestment'].max(),20)
    ax.scatter(sorted_df['marketingInvestment'], sorted_df['netRevenue'])
    ax.plot(x,log_f(x, *regr))
    ax.set_xlabel(f'Investment {name[1]}')
    ax.set_ylabel(f'Revenue {name[1]}')
    ax.set_title(name[0])
    plt.savefig( name[0] + '.jpg', dpi=150, pad_inches=0.1)  
    plt.close()
    #plt.show()


In [359]:
df = pd.read_csv('input\\'+ list_files[15],
                 sep="\t",
                 infer_datetime_format=True,
                 usecols=usecols,
                 dtype=dtype,
                 parse_dates=DateTimeID)

df['periodStartDate'] = pd.to_datetime(df['periodStartDate'])

#df.head(5)

In [360]:
df.shape

(1030712, 16)

In [361]:
#list_files[6]

In [362]:
platform_code = df['platformCode'].unique().tolist()
platform_code.sort()
business_unit_code = df['businessUnit'].unique().tolist()
business_unit_code.sort()
channel_code_code = df['channelCode'].unique().tolist()
channel_code_code.sort()
currency_code_code = df['currencyCode'].unique().tolist()
currency_code_code.sort()
account_code = df['account'].unique().tolist()


In [363]:
account_code

['superzoo']

In [364]:
#select window for regression 
excel_coll = 0
precision = 3

workbook = xlsxwriter.Workbook(folder + account_code[0]+'_corr.xlsx')
bold = workbook.add_format({'bold': True})


df_time_selected = df[df['periodStartDate'] >= df['periodStartDate'].max() - pd.DateOffset(weeks=time_window_regr)] 

#df_time_selected = filter_df(df_time_selected, 0.1)

for i in business_unit_code:
    df_selected = df_time_selected[df_time_selected['businessUnit'] == i]
    worksheet = workbook.add_worksheet(i)
    
    worksheet.set_column('A:A', 30)
    worksheet.set_column('B:B', 15)
    worksheet.set_column('C:C', 15)
    worksheet.set_column('D:D', 15)
    worksheet.set_column('E:E', 15)
    worksheet.set_column('F:F', 15)
    worksheet.set_column('G:G', 40)

    worksheet.write('A1', 'Name', bold)
    worksheet.write('B1', 'Spearman corr', bold)
    worksheet.write('C1', 'a', bold)
    worksheet.write('D1', 'b', bold)
    worksheet.write('E1', 'c', bold)
    worksheet.write('F1', 'none', bold)
    worksheet.write('G1', 'Chart', bold)
    
    excel_row  = 2
    
    for z in platform_code:
        df_platform = df_selected[df_selected['platformCode'] == z]
        df_business_selected_agg = select_agg_resample_df(df_platform, 'periodStartDate', 'week', use_nan = False)
        sper_crr, _=stat.spearmanr(df_business_selected_agg['marketingInvestment'], df_business_selected_agg['netRevenue'])
        if sper_crr> 0.75:
            print (i + ' business unit '+ z + ' platform has \t'+ str(sper_crr) + ' correlation coefficient')

            try:
                sorted_df = df_business_selected_agg.sort_values( by=['marketingInvestment'], ascending=False )
                t_train = sorted_df['marketingInvestment']
                y_train = sorted_df['netRevenue']
                
                regr, err = regression_calc(sorted_df[['marketingInvestment','netRevenue']],log_f)
                ing_name = [i+z, currency_code_code]
                save_chart(sorted_df, ing_name)
                
                a,b,c = regr
                #print(a,b,c)
                worksheet.write('A'+str(excel_row), z)
                worksheet.write('B'+str(excel_row), str(np.round(sper_crr, precision)))
                
                worksheet.write('C'+str(excel_row), str(np.round(a, precision)))
                worksheet.write('D'+str(excel_row), str(np.round(b, precision)))
                worksheet.write('E'+str(excel_row), str(np.round(c, precision)))
                worksheet.insert_image('G'+ str(excel_row), i + z + '.jpg', {'x_scale': 0.5, 'y_scale': 0.5})
                
                excel_row = excel_row + 10
                print('Regression calculated successfully')
            except:
                print("Regression wasn't calculated")
                worksheet.write('A'+ str(excel_row), z)
                excel_row = excel_row + 1

        else:
            pass

workbook.close()

ontariocz business unit AdWords platform has 	0.8626373626373626 correlation coefficient
Regression calculated successfully
superzoocz business unit CjAffiliate platform has 	0.9725274725274725 correlation coefficient
Regression calculated successfully
superzoocz business unit HeurekaCz platform has 	0.7912087912087913 correlation coefficient
Regression calculated successfully
superzoosk business unit AdWords platform has 	0.8571428571428571 correlation coefficient
Regression calculated successfully


In [365]:
#df_business_selected_agg['netRevenue'].plot()

In [258]:
#df_business_selected_agg['marketingInvestment'].plot()

In [254]:
df_business_selected_agg_filtered = filter_df(df_business_selected_agg, 0.1)
#df_business_selected_agg_filtered['marketingInvestment'].plot()

In [255]:

spermn_corr = stat.spearmanr(df_business_selected_agg['marketingInvestment'], df_business_selected_agg['netRevenue'])

In [22]:
spermn_corr

SpearmanrResult(correlation=nan, pvalue=nan)

In [23]:
sper_corr, _ = spermn_corr
sper_corr

nan

In [27]:
df_business_selected = df[df['businessUnit'] == 'kytarycz']
df_business_selected = df_business_selected[df_business_selected['platformCode'] == 'AdWords']

df_business_selected_agg = select_agg_resample_df(df_business_selected, 'periodStartDate', 'week', use_nan = False)

sorted_df = df_business_selected_agg.sort_values( by=['marketingInvestment'], ascending=False )
t_train = sorted_df['marketingInvestment']
y_train = sorted_df['netRevenue']

try:
    regr, err = regression_calc(sorted_df[['marketingInvestment','netRevenue']],log_f)
except:
    print("Regression wasn't calculated")

save_chart(sorted_df,['Hello','CZK'] )

In [28]:
regr

array([-550539.41360935, -112226.06432739,  205807.23840565])

In [ ]:
R = sorted_df['netRevenue'] - log_f(sorted_df['marketingInvestment'], *popt)

In [ ]:
plt.scatter(sorted_df['marketingInvestment'], R)

In [ ]:
sorted_df['netRevenue'].describe()[2]

In [ ]:
R.describe()[2]

In [ ]:
statistics_R2 = []

statistics_R2.append(np.concatenate((regr, err), axis=0))



statistics_R2_df = pd.DataFrame(statistics_R2, columns = regr_name)
statistics_R2_df = statistics_R2_df.set_index([pd.Index(platform_code)])

In [ ]:
#log_f(sorted_df['marketingInvestment'], *popt)